In [46]:
import openai
from dotenv import load_dotenv
import os
import json
import numpy as np

load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_API_KEY")

from benlp.llms import Chat

In [6]:
chat = Chat(model='gpt-3.5-turbo-0613')
res = chat("This is a test. Tell me a long story.")['response']

In [15]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Step 1, send model the user query and what functions it has access to
def run_conversation():
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    # Step 2, check if the model wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        # Step 3, call the function
        # Note: the JSON response from the model may not be valid JSON
        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )

        # Step 4, send model the info on the function call and function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "What is the weather like in boston?"},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print(run_conversation())

{
  "id": "chatcmpl-7R5wOL7HclUgAhv8kKs3K4H6BVNot",
  "object": "chat.completion",
  "created": 1686692048,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston is sunny and windy with a temperature of 72 degrees."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 67,
    "completion_tokens": 17,
    "total_tokens": 84
  }
}


In [37]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )

In [38]:
response

<OpenAIObject chat.completion id=chatcmpl-7R6FM0QbUSRH5oaKJCQO4xw7mQHpt at 0x7fe11fe79e50> JSON: {
  "id": "chatcmpl-7R6FM0QbUSRH5oaKJCQO4xw7mQHpt",
  "object": "chat.completion",
  "created": 1686693224,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}

## My first implementation

My custom function

In [24]:
from scipy.optimize import linprog

def linear_optimization_simplex(c, A_ub, b_ub, A_eq=None, b_eq=None, bounds=None):
    """
    Solves a linear programming optimization problem using the simplex algorithm.

    Parameters:
    c (list): The coefficients of the linear objective function to be minimized.
    A_ub (list): The inequality constraint matrix. Each row represents a constraint.
    b_ub (list): The inequality constraint vector. Each element represents the upper bound of the corresponding constraint.
    A_eq (list, optional): The equality constraint matrix. Each row represents a constraint.
    b_eq (list, optional): The equality constraint vector. Each element represents the required value of the corresponding constraint.
    bounds (list, optional): A list of bounds for each variable in the form (min, max).

    Returns:
    dict: A dictionary containing the optimal solution, the minimized objective function value, and the status of the optimization.
    """
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')
    
    return {
        'x': result.x,
        'fun': result.fun,
        'success': result.success,
        'message': result.message
    }

In [93]:

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "Solve the following linear programming problem using the simplex algorithm: Minimize the objective function -1x + 4y subject to the constraints -3x + y <= 6, x + 2y <= 4, and y >= -3."}],
    functions=[
        {
            "name": "linear_optimization_simplex",
            "description": "Solves a linear programming optimization problem using the simplex algorithm.",
            "parameters": {
                "type": "object",
                "properties": {
                    "c": {
                        "type": "string",
                        "description": "The coefficients of the linear objective function to be minimized.",
                    },
                    "A_ub": {
                        "type": "string",
                        "description": "The inequality constraint matrix. Each row represents a constraint.",
                    },
                    "b_ub": {
                        "type": "string",
                        "description": "The inequality constraint vector. Each element represents the upper bound of the corresponding constraint.",
                    },
                    "A_eq": {
                        "type": "string",
                        "description": "The equality constraint matrix. Each row represents a constraint.",
                    },
                    "b_eq": {
                        "type": "string",
                        "description": "The equality constraint vector. Each element represents the required value of the corresponding constraint.",
                    },
                    "bounds": {
                        "type": "string",
                        "description": "A list of bounds for each variable in the form (min, max).",
                    },
                },
                "required": ["c", "A_ub", "b_ub"],
            },
        }
    ],
    function_call="auto",
)

In [44]:
response

<OpenAIObject chat.completion id=chatcmpl-7R6GZKdKM4jOwVTof95OdsfKqB8vA at 0x7fe11f704c70> JSON: {
  "id": "chatcmpl-7R6GZKdKM4jOwVTof95OdsfKqB8vA",
  "object": "chat.completion",
  "created": 1686693299,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "linear_optimization_simplex",
          "arguments": "{\n  \"c\": \"-1 4\",\n  \"A_ub\": \"-3 1 ; 1 2\",\n  \"b_ub\": \"6 4\",\n  \"A_eq\": \"0 1\",\n  \"b_eq\": \"-3\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 216,
    "completion_tokens": 63,
    "total_tokens": 279
  }
}

In [45]:
message = response["choices"][0]["message"]
message

<OpenAIObject at 0x7fe11fe799f0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "linear_optimization_simplex",
    "arguments": "{\n  \"c\": \"-1 4\",\n  \"A_ub\": \"-3 1 ; 1 2\",\n  \"b_ub\": \"6 4\",\n  \"A_eq\": \"0 1\",\n  \"b_eq\": \"-3\"\n}"
  }
}

In [69]:
def parse_json_object(json_object):
    function_call = json_object["function_call"]
    arguments_str = function_call["arguments"]
    arguments = json.loads(arguments_str)
    
    c = np.fromstring(arguments.get("c", ""), sep=" ")
    A_ub = np.vstack([np.fromstring(row, sep=" ") for row in arguments.get("A_ub", "").split(";")]) if arguments.get("A_ub") else np.array([])
    b_ub = np.fromstring(arguments.get("b_ub", ""), sep=" ")
    A_eq = np.vstack([np.fromstring(row, sep=" ") for row in arguments.get("A_eq", "").split(";")]) if arguments.get("A_eq") else np.array([])
    b_eq = np.fromstring(arguments.get("b_eq", ""), sep=" ")
    bounds = [tuple(map(float, bound.split(","))) for bound in arguments.get("bounds", "").split(";")] if arguments.get("bounds") else None
    
    return c, A_ub, b_ub, A_eq, b_eq, bounds

In [62]:

if message.get("function_call"):
    function_name = message["function_call"]["name"]
    # Parse the arguments
    c, A_ub, b_ub, A_eq, b_eq = parse_json_object(message)
    print("c:", c)
    print("A_ub:", A_ub)
    print("b_ub:", b_ub)
    print("A_eq:", A_eq)
    print("b_eq:", b_eq)
    # Call the linear_optimization_simplex function with the parsed arguments
    function_response = linear_optimization_simplex(c, A_ub, b_ub, A_eq, b_eq)
    print(function_response)

    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": "Solve the following linear programming problem using the simplex algorithm: Minimize the objective function -1x + 4y subject to the constraints -3x + y <= 6, x + 2y <= 4, and y >= -3."},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(str(function_response))
            },
        ],
    )

    print(second_response)

c: [-1.  4.]
A_ub: [[-3.  1.]
 [ 1.  2.]]
b_ub: [6. 4.]
A_eq: [[0. 1.]]
b_eq: [-3.]
{'x': array([0., 0.]), 'fun': 0.0, 'success': False, 'message': 'The problem is (trivially) infeasible because a singleton row in the equality constraints is inconsistent with the bounds.'}


/var/folders/6t/m1cj5jqd1cx9xq5bxs_h3g200000gn/T/ipykernel_28558/3480648203.py:18: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='simplex')


{
  "id": "chatcmpl-7R6Upw7OOZ7WIZob1On5mMu4SUj6n",
  "object": "chat.completion",
  "created": 1686694183,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The linear programming problem is infeasible because a singleton row in the equality constraints is inconsistent with the bounds."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 188,
    "completion_tokens": 22,
    "total_tokens": 210
  }
}


In [91]:
def run_chat(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": prompt}],
        functions=[
            {
                "name": "linear_optimization_simplex",
                "description": "Solves a linear programming optimization problem using the simplex algorithm.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "c": {
                            "type": "AnyOf",
                            "description": "The coefficients of the linear objective function to be minimized.",
                        },
                        "A_ub": {
                            "type": "string",
                            "description": "The inequality constraint matrix. Each row represents a constraint.",
                        },
                        "b_ub": {
                            "type": "string",
                            "description": "The inequality constraint vector. Each element represents the upper bound of the corresponding constraint.",
                        },
                        "A_eq": {
                            "type": "string",
                            "description": "The equality constraint matrix. Each row represents a constraint.",
                        },
                        "b_eq": {
                            "type": "string",
                            "description": "The equality constraint vector. Each element represents the required value of the corresponding constraint.",
                        },
                        "bounds": {
                            "type": "string",
                            "description": "A list of bounds for each variable in the form (min, max).",
                        },
                    },
                    "required": ["c", "A_ub", "b_ub"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]
    print("MESSAGE 1", message)

    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        arg_dict = json.loads(message["function_call"]["arguments"])

        validated_args = {
            "c": np.array(f"[{arg_dict.get('c', '')}]"),
            "A_ub": np.array(f"[{arg_dict.get('A_ub', '')}]"),
            "b_ub": np.array(f"[{arg_dict.get('b_ub', '')}]"),
            "A_eq": np.array(f"[{arg_dict.get('A_eq', '')}]") if arg_dict.get("A_eq") else None,
            "b_eq": np.array(f"[{arg_dict.get('b_eq', '')}]") if arg_dict.get("b_eq") else None,
            "bounds": np.array(f"[{arg_dict.get('bounds', '')}]") if arg_dict.get("bounds") else None,
        }
        
        function_response = linear_optimization_simplex(**validated_args)
        print(function_response)

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": "Solve the following linear programming problem using the simplex algorithm: Minimize the objective function -1x + 4y subject to the constraints -3x + y <= 6, x + 2y <= 4, and y >= -3."},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": json.dumps(str(function_response))
                },
            ],
        )

        return second_response

In [64]:
test_problem = "A factory produces two types of products, A and B. Each unit of product A requires 2 hours of labor and 3 hours of machine time, while each unit of product B requires 4 hours of labor and 1 hour of machine time. The factory has a maximum of 40 hours of labor and 30 hours of machine time available per day. The profit per unit of product A is $5, and the profit per unit of product B is $4. How many units of each product should the factory produce to maximize its profit?"

In [92]:
res = run_chat(test_problem)

InvalidRequestError: Invalid schema for function 'linear_optimization_simplex': 'AnyOf' is not valid under any of the given schemas